In [1]:
import sys
sys.path.insert(0, "../")
from spot_master.unet import data
from spot_master.unet.model import UNet
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
from torchvision.transforms import Compose

/opt/anaconda/envs/cuda/lib/python3.10/site-packages/tqdm-4.65.0-py3.10.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_transforms = Compose([
    data.RandomHorizontalFlip(),
    data.RandomRotation(),
    data.ToTensorWrapper(),
])

In [3]:
train_dataset = data.FISHSpotsDataset(
    meta_csv="meta_train.csv", root_dir="../FISH_spots", transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

test_dataset = data.FISHSpotsDataset(
    meta_csv="meta_test.csv", root_dir="../FISH_spots")
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(1, 1, 4).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [9]:
# TensorBoard
writer = SummaryWriter("runs/fish_spots_experiment")

In [10]:
# Train
num_epochs = 50
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for idx, batch in enumerate(train_loader):
        images = batch["image"].to(device, dtype=torch.float)
        masks = batch["mask"].to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        if idx % 10 == 0:
            print(f"Epoch: {epoch + 1}/{num_epochs}, Batch: {idx + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")
            writer.add_scalar("Loss/train_batch", loss.item(), epoch * len(train_loader) + idx)

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    writer.add_scalar("Loss/train", epoch_loss, epoch)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            images = batch["image"].to(device, dtype=torch.float)
            masks = batch["mask"].to(device, dtype=torch.float)

            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    writer.add_scalar("Loss/val", val_loss, epoch)

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print(f"Best model saved with Val Loss: {val_loss:.4f}")

writer.close()

Epoch: 1/50, Batch: 1/269, Loss: 0.8230
Epoch: 1/50, Batch: 11/269, Loss: 0.8177
Epoch: 1/50, Batch: 21/269, Loss: 0.8171
Epoch: 1/50, Batch: 31/269, Loss: 0.8166
Epoch: 1/50, Batch: 41/269, Loss: 0.8160
Epoch: 1/50, Batch: 51/269, Loss: 0.8155
Epoch: 1/50, Batch: 61/269, Loss: 0.8149
Epoch: 1/50, Batch: 71/269, Loss: 0.8144
Epoch: 1/50, Batch: 81/269, Loss: 0.8138
Epoch: 1/50, Batch: 91/269, Loss: 0.8133
Epoch: 1/50, Batch: 101/269, Loss: 0.8127
Epoch: 1/50, Batch: 111/269, Loss: 0.8121
Epoch: 1/50, Batch: 121/269, Loss: 0.8116
Epoch: 1/50, Batch: 131/269, Loss: 0.8110
Epoch: 1/50, Batch: 141/269, Loss: 0.8105
Epoch: 1/50, Batch: 151/269, Loss: 0.8099
Epoch: 1/50, Batch: 161/269, Loss: 0.8094
Epoch: 1/50, Batch: 171/269, Loss: 0.8088
Epoch: 1/50, Batch: 181/269, Loss: 0.8083
Epoch: 1/50, Batch: 191/269, Loss: 0.8077
Epoch: 1/50, Batch: 201/269, Loss: 0.8072
Epoch: 1/50, Batch: 211/269, Loss: 0.8066
Epoch: 1/50, Batch: 221/269, Loss: 0.8061
Epoch: 1/50, Batch: 231/269, Loss: 0.8055
Epo

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/anaconda/envs/cuda/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/anaconda/envs/cuda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/anaconda/envs/cuda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/wzxu/Projects/SpotMaster/notebooks/../spot_master/unet/data.py", line 31, in __getitem__
    image = io.imread(img_path)
  File "/opt/anaconda/envs/cuda/lib/python3.10/site-packages/skimage/io/_io.py", line 53, in imread
    img = call_plugin('imread', fname, plugin=plugin, **plugin_args)
  File "/opt/anaconda/envs/cuda/lib/python3.10/site-packag